In [ ]:
import json 
import os 
import dashscope
from dashscope.api_entities.dashscope_response import Role

In [ ]:
import torch
print(torch.__version__)              # 应该能输出版本号
print(hasattr(torch.nn, 'Module'))    # 应该输出 True


In [ ]:
from modelscope import snapshot_download

model_dir = snapshot_download(
    repo_id='deepseek-ai/DeepSeek-R1-Distill-Qwen-1.5B',        # Hub 上的模型标识
    cache_dir='/Users/zengguangyun/Desktop/data/大模型课程资料/autodl-tmp/models'  # 本地存储根目录
)
print(f"模型已下载到：{model_dir}")


In [ ]:
import torch
print(torch.__version__)                  # 打印版本号
print(torch.backends.mps.is_available())  # True 表示能用 MPS


In [ ]:
import torch
from modelscope import AutoModelForCausalLM, AutoTokenizer

model_name = "/Users/zengguangyun/Desktop/data/大模型课程资料/autodl-tmp/models/deepseek-ai/DeepSeek-R1-Distill-Qwen-1.5B"

# 先确定用哪个设备
device = torch.device("mps") if torch.backends.mps.is_available() else torch.device("cpu")

# 加载模型（不传 device_map）
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    torch_dtype="auto",
)
model.to(device)  # 搬到 MPS 或 CPU

tokenizer = AutoTokenizer.from_pretrained(model_name)

prompt = "帮我写一个二分查找法"
messages = [
    {"role": "system", "content": "You are a helpful assistant."},
    {"role": "user",   "content": prompt}
]
text = tokenizer.apply_chat_template(
    messages,
    tokenize=False,
    add_generation_prompt=True
)
inputs = tokenizer([text], return_tensors="pt").to(device)

generated_ids = model.generate(
    **inputs,
    max_new_tokens=2000
)
# 切掉输入部分再解码
response_ids = generated_ids[0][ inputs.input_ids.shape[-1]: ]
print(tokenizer.decode(response_ids, skip_special_tokens=True))
